<a href="https://colab.research.google.com/github/ClaFlorez/Projet_Panda_POO/blob/main/Copy_of_Projet_Panda_POO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# Projet de Data Science : Analyse de Données de Ventes avec Pandas et POO

**Objectif du projet :** Ce projet vise à appliquer les compétences acquises en manipulation de données avec la bibliothèque Pandas en Python. Vous mettrez en pratique le chargement, le nettoyage, la transformation, l'analyse et la combinaison de données dans un contexte réaliste d'analyse de ventes d'une entreprise e-commerce.

**Contrainte principale :** L'ensemble de la logique d'analyse devra être encapsulé dans une classe Python (Programmation Orientée Objet), afin de structurer le code de manière professionnelle et réutilisable.

## Contexte

Vous êtes analyste de données pour une entreprise de vente en ligne. Votre manager vous a confié deux fichiers de données :

1.  `commandes.csv` : Contient les informations sur chaque commande passée.
2.  `clients.csv` : Contient les détails sur les clients enregistrés.

Votre mission est de construire une classe `AnalyseVentes` qui prendra en charge l'ensemble du processus d'analyse, depuis le chargement des données brutes jusqu'à la production d'indicateurs clés pour la direction.

## Les Données

Voici une description des colonnes attendues dans chaque fichier :

**`commandes.csv`**
- `id_commande`: Identifiant unique de la commande.
- `id_client`: Identifiant du client ayant passé la commande (clé de jointure).
- `date_commande`: Date de la commande (format `AAAA-MM-JJ`).
- `produit`: Nom du produit acheté.
- `quantite`: Nombre d'unités achetées.
- `prix_unitaire`: Prix d'une unité du produit (peut contenir des symboles monétaires comme '$').

**`clients.csv`**
- `id_client`: Identifiant unique du client.
- `nom`: Nom du client.
- `email`: Adresse e-mail du client.
- `ville`: Ville de résidence du client.

## Étape 1 : Mise en Place de l'Environnement et de la Classe

Importez la bibliothèque Pandas. Ensuite, créez la structure de base de votre classe `AnalyseVentes`. Cette classe sera le conteneur pour toutes vos méthodes d'analyse. Le constructeur `__init__` devra accepter les chemins vers les deux fichiers CSV.

In [ ]:
import pandas as pd
import numpy as np # Import numpy here as it will be used later

class AnalyseVentes:
    def __init__(self, chemin_commandes, chemin_clients):
        """Constructeur de la classe.
        Charge les datasets depuis les chemins spécifiés et les stocke en tant qu'attributs.
        Astuce : Utilisez un bloc try-except pour gérer les erreurs de chargement de fichier."""
        self.chemin_clients = chemin_clients
        self.chemin_commandes = chemin_commandes
        try:
            self.clients = pd.read_csv(chemin_clients)
            print("Clients.csv cargado correctamente.")
        except Exception as e:
            print("Error al cargar clients.csv:", e)
            raise e
        try:
            self.commandes = pd.read_csv(chemin_commandes)
            print("Commandes.csv cargado correctamente.")
        except Exception as e:
            print("Error al cargar commandes.csv:", e)
            raise e

        # Attribut pour le DataFrame fusionné
        self.data_complet = None

    def nettoyer_donnees(self):
        """Méthode principale qui appelle les autres fonctions de nettoyage."""
        self._convertir_types()
        self._gerer_valeurs_manquantes()
        self._supprimer_doublons_clients() # Corrected method name
        print("Nettoyage des données terminé.")

    def _convertir_types(self): # Made private as it's called internally
        self.commandes['date_commande'] = pd.to_datetime(self.commandes['date_commande'], errors='coerce')
        self.commandes['prix_unitaire'] = self.commandes['prix_unitaire'].astype(str).str.replace('$', '', regex=False)
        self.commandes['prix_unitaire'] = pd.to_numeric(self.commandes['prix_unitaire'], errors='coerce')
        self.commandes['quantite'] = pd.to_numeric(self.commandes['quantite'], errors='coerce')
        print("Types de données convertis avec succès.")


    def _gerer_valeurs_manquantes(self): # Made private as it's called internally
        print("Résumé des valeurs manquantes dans commandes :")
        print(self.commandes.isnull().sum())
        self.commandes.dropna(subset=['quantite', 'prix_unitaire', 'date_commande'], inplace=True)
        print("Valeurs manquantes gérées (quantite, prix_unitaire, date_commande).")


    def _supprimer_doublons_clients(self): # Made private and corrected method name
        avant = len(self.clients)
        self.clients.drop_duplicates(subset='id_client', inplace=True)
        apres = len(self.clients)
        print(f"Doublons supprimés dans clients : {avant - apres} ligne(s) supprimée(s).")

    def fusionner_donnees(self):
        """Fusionne les dataframes commandes et clients sur id_client."""
        self.data_complet = pd.merge(self.commandes, self.clients, on='id_client', how='inner')
        print("Fusion des données terminée.")


    def enrichir_donnees(self):
        """Ajoute les colonnes montant_total, categorie_client et mois_commande."""
        if self.data_complet is None:
            print("Veuillez d'abord fusionner les données.")
            return
        self.data_complet['montant_total'] = self.data_complet['quantite'] * self.data_complet['prix_unitaire']
        self.data_complet['categorie_client'] = np.where(self.data_complet['montant_total'] > 1000, 'Or',
                                                          np.where(self.data_complet['montant_total'] > 500, 'Argent', 'Bronze'))
        self.data_complet['mois_commande'] = self.data_complet['date_commande'].dt.month
        print("Enrichissement des données terminé.")

    def rapport_general(self):
        """Affiche les informations générales et les statistiques descriptives du dataframe complet."""
        if self.data_complet is None:
            print("Veuillez d'abord fusionner les données.")
            return
        print("\n--- Info sur le DataFrame complet ---")
        self.data_complet.info()
        print("\n--- Statistiques descriptives ---")
        display(self.data_complet.describe(include='all'))


    def chiffre_affaires_par_ville(self):
        """Calcule le chiffre d'affaires total par ville et le retourne trié."""
        if self.data_complet is None:
            print("Veuillez d'abord fusionner les données.")
            return
        ca_ville = self.data_complet.groupby('ville')['montant_total'].sum().sort_values(ascending=False)
        return ca_ville

    def top_5_clients(self):
        """Identifie le top 5 des clients par chiffre d'affaires total."""
        if self.data_complet is None:
            print("Veuillez d'abord fusionner les données.")
            return
        top_clients = self.data_complet.groupby(['id_client', 'nom'])['montant_total'].sum().reset_index()
        top_clients = top_clients.sort_values(by='montant_total', ascending=False).head(5)
        return top_clients

    def ventas_mensuales_par_producto(self):
        """Crea una tabla dinámica de las ventas mensuales por producto."""
        if self.data_complet is None:
            print("Veuillez d'abord fusionner les données.")
            return
        pivot_table = pd.pivot_table(self.data_complet, values='montant_total', index='produit', columns='mois_commande', aggfunc='sum', fill_value=0)
        return pivot_table

In [ ]:
url_clients = "https://raw.githubusercontent.com/ClaFlorez/Projet_Panda_POO/main/clients.csv"
url_commandes = "https://raw.githubusercontent.com/ClaFlorez/Projet_Panda_POO/main/commandes.csv"

analyse = AnalyseVentes(url_commandes, url_clients)


Clients.csv cargado correctamente.
Commandes.csv cargado correctamente.


## Étape 2 : Nettoyage et Préparation des Données (Data Cleaning)

Ajoutez des méthodes à votre classe pour effectuer les tâches de nettoyage suivantes. Chaque tâche doit être implémentée dans sa propre méthode pour une meilleure clarté.

1.  **Gestion des types de données :**
    - La colonne `date_commande` doit être convertie au format `datetime`.
    - La colonne `prix_unitaire` contient des '$' et doit être convertie en type numérique (`float`). Utilisez la méthode `.apply()` avec une fonction lambda pour supprimer le symbole et convertir le type.
2.  **Gestion des valeurs manquantes :**
    - Affichez un résumé des valeurs manquantes.
    - Supprimez les lignes où la `quantite` est manquante dans le DataFrame des commandes.
3.  **Gestion des doublons :**
    - Vérifiez et supprimez les éventuels doublons dans le DataFrame des clients basé sur l'`id_client`.

In [ ]:
# Ajoutez ici les méthodes de nettoyage à votre classe AnalyseVentes

# Exemple de squelette pour une méthode
class AnalyseVentes(AnalyseVentes): # On "étend" la classe déjà définie
    def nettoyer_donnees(self):
        """Méthode principale qui appelle les autres fonctions de nettoyage."""
        self._convertir_types()
        self._gerer_valeurs_manquantes()
        self._supprimer_doublons()
        print("Nettoyage des données terminé.")

    def convertir_types(self):
        self.commandes['date_commande'] = pd.to_datetime(self.commandes['date_commande'], errors='coerce')
        self.commandes['prix_unitaire'] = self.commandes['prix_unitaire'].apply(lambda x: float(str(x).replace('$', '')))
        print("Types de données convertis avec succès.")
        pass
    def gerer_valeurs_manquantes(self):
        print("Résumé des valeurs manquantes dans commandes :")
        print(self.commandes.isnull().sum())
        self.commandes.dropna(subset=['quantite'], inplace=True)
        print("Valeurs manquantes gérées (quantite).")
        pass

    def supprimer_doublons_clients(self):
        avant = len(self.clients)
        self.clients.drop_duplicates(subset='id_client', inplace=True)
        apres = len(self.clients)
        print(f"Doublons supprimés dans clients : {avant - apres} ligne(s) supprimée(s).")
        pass

In [ ]:
# analyse.nettoyer_donnees() # This line is redundant and causing the error
# analyse.convertir_types() # These methods are now called within nettoyer_donnees
# analyse.gerer_valeurs_manquantes()
# analyse.supprimer_doublons()

## Étape 3 : Fusion et Enrichissement des Données

1.  **Fusionner les DataFrames :**
    - Créez une méthode `fusionner_donnees` qui effectue une jointure (`pd.merge`) entre le DataFrame des commandes et celui des clients sur la colonne `id_client`. Le résultat doit être stocké dans l'attribut `self.data_complet`.
2.  **Création de nouvelles colonnes (Feature Engineering) :**
    - Créez une méthode `enrichir_donnees`.
    - Ajoutez une colonne `montant_total` qui est le produit de `quantite` et `prix_unitaire`.
    - En utilisant une **colonne conditionnelle** (avec `np.where` ou `.loc`), créez une colonne `categorie_client` : 'Or' si le montant total de la commande > 1000, 'Argent' si > 500, et 'Bronze' sinon.
    - **Catégorisation :** Créez une colonne `mois_commande` en extrayant le mois de la `date_commande` (`.dt.month`).

In [ ]:
# Ajoutez ici les méthodes de fusion et d'enrichissement à votre classe
import numpy as np

class AnalyseVentes(AnalyseVentes):
    def fusionner_donnees(self):
        # À COMPLÉTER : Fusionner self.commandes_df et self.clients_df
        # Le résultat est stocké dans self.data_complet
        print("Fusion des données terminée.")
        pass

    def enrichir_donnees(self):
        if self.data_complet is None:
            print("Veuillez d'abord fusionner les données.")
            return
        # À COMPLÉTER : Créer la colonne 'montant_total'
        # À COMPLÉTER : Créer la colonne 'categorie_client' avec une condition
        # À COMPLÉTER : Créer la colonne 'mois_commande'
        print("Enrichissement des données terminé.")
        pass

## Étape 4 : Analyse Statistique et Tri

Implémentez des méthodes pour répondre aux questions business suivantes. Chaque méthode doit retourner un DataFrame ou une Série Pandas contenant le résultat.

1.  **Statistiques descriptives :** Créez une méthode `rapport_general` qui affiche les résultats de `.info()` et `.describe()` sur le jeu de données complet.
2.  **Chiffre d'affaires par ville :** Calculez le chiffre d'affaires total (somme de `montant_total`) par ville. Triez les résultats par ordre décroissant.
3.  **Meilleurs clients :** Identifiez le top 5 des clients en termes de chiffre d'affaires total. La méthode doit retourner un DataFrame avec `id_client`, `nom` et `montant_total`.
4.  **Analyse mensuelle :** Créez une **table pivot** (`pivot_table`) qui montre le chiffre d'affaires total pour chaque `produit` (en lignes) et chaque `mois_commande` (en colonnes).

In [ ]:
# Ajoutez ici les méthodes d'analyse à votre classe
class AnalyseVentes(AnalyseVentes):
    def rapport_general(self):
        # À COMPLÉTER : Afficher les infos et les descriptions statistiques
        pass

    def chiffre_affaires_par_ville(self):
        # À COMPLÉTER : Utiliser .groupby() pour regrouper par 'ville', calculer la somme de 'montant_total'
        # et trier les résultats avec .sort_values()
        # La méthode doit retourner le DataFrame résultant.
        pass

    def top_5_clients(self):
        # À COMPLÉTER : Grouper par client, sommer le montant_total, trier et prendre les 5 premiers.
        pass

    def ventes_mensuelles_par_produit(self):
        # À COMPLÉTER : Créer une table pivot
        pass

## Étape 5 : Exécution du Pipeline d'Analyse

Maintenant que votre classe est complète, instanciez-la et appelez les méthodes dans le bon ordre pour effectuer l'analyse complète.

In [ ]:
# --- Exécution Principale ---

# Chemins vers vos fichiers (assurez-vous qu'ils existent)
path_cde = 'https://raw.githubusercontent.com/ClaFlorez/Projet_Panda_POO/main/commandes.csv'
path_cli = 'https://raw.githubusercontent.com/ClaFlorez/Projet_Panda_POO/main/clients.csv'

# 1. Instanciation de la classe
analyseur = AnalyseVentes(path_cde, path_cli)

# 2. Exécution du nettoyage
analyseur.nettoyer_donnees()

# 3. Fusion des données
analyseur.fusionner_donnees()

# 4. Enrichissement
analyseur.enrichir_donnees()

# 5. Génération des rapports d'analyse
print("\n--- Rapport Général ---")
analyseur.rapport_general()

print("\n--- Chiffre d'affaires par ville ---")
ca_ville = analyseur.chiffre_affaires_par_ville()
display(ca_ville)

print("\n--- Top 5 Clients ---")
top_clients = analyseur.top_5_clients()
display(top_clients)

print("\n--- Ventes Mensuelles par Produit ---")
pivot = analyseur.ventas_mensuales_par_producto()
display(pivot)

Clients.csv cargado correctamente.
Commandes.csv cargado correctamente.
Types de données convertis avec succès.
Résumé des valeurs manquantes dans commandes :
id_commande       0
id_client         0
date_commande     0
produit           0
quantite         47
prix_unitaire     0
dtype: int64
Valeurs manquantes gérées (quantite, prix_unitaire, date_commande).
Doublons supprimés dans clients : 2 ligne(s) supprimée(s).
Nettoyage des données terminé.
Fusion des données terminée.
Enrichissement des données terminé.

--- Rapport Général ---

--- Info sur le DataFrame complet ---
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 453 entries, 0 to 452
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   id_commande       453 non-null    int64         
 1   id_client         453 non-null    int64         
 2   date_commande     453 non-null    datetime64[ns]
 3   produit           453 non-null    object 

,id_commande,id_client,date_commande,produit,quantite,prix_unitaire,nom,email,ville,montant_total,categorie_client,mois_commande
count,453.000000,453.000000,453,453,453.000000,453.000000,453,453,453,453.000000,453,453.000000
unique,NaN,NaN,NaN,5,NaN,NaN,96,96,92,NaN,3,NaN
top,NaN,NaN,NaN,Clavier Mécanique,NaN,NaN,Martine Bouvier,chretieneugene@example.org,Diallo,NaN,Bronze,NaN
freq,NaN,NaN,NaN,108,NaN,NaN,11,11,15,NaN,269,NaN
mean,1249.856512,50.068433,2024-07-03 16:54:02.384105984,NaN,1.445916,487.110883,NaN,NaN,NaN,727.771567,NaN,6.437086
min,1001.000000,1.000000,2023-06-30 00:00:00,NaN,1.000000,75.000000,NaN,NaN,NaN,75.000000,NaN,1.000000
25%,1127.000000,23.000000,2024-01-08 00:00:00,NaN,1.000000,99.990000,NaN,NaN,NaN,150.000000,NaN,4.000000
50%,1250.000000,50.000000,2024-06-28 00:00:00,NaN,1.000000,150.250000,NaN,NaN,NaN,225.000000,NaN,6.000000
75%,1372.000000,77.000000,2025-01-05 00:00:00,NaN,2.000000,850.990000,NaN,NaN,NaN,1200.500000,NaN,9.000000
max,1499.000000,100.000000,2025-06-29 00:00:00,NaN,3.000000,1200.500000,NaN,NaN,NaN,3601.500000,NaN,12.000000



--- Chiffre d'affaires par ville ---


,montant_total
ville,
Merle,10659.38
Courtois-les-Bains,9906.22
Bonneau,9155.69
Diallo,9031.21
Verdier-la-Forêt,8805.71
...,...
Gallet-les-Bains,399.96
BarthelemyVille,199.98
GrosVille,199.98



--- Top 5 Clients ---


,id_client,nom,montant_total
80,83,Nathalie Bailly,9906.22
34,36,Martine Bouvier,9155.69
36,38,Catherine Le Roux-Payet,8805.71
66,69,Franck Renault,8128.99
49,51,Thomas-Augustin De Oliveira,7804.47



--- Ventes Mensuelles par Produit ---


mois_commande,1,2,3,4,5,6,7,8,9,10,11,12
produit,,,,,,,,,,,,
Clavier Mécanique,1953.25,1652.75,1953.25,1953.25,1352.25,2103.50,1652.75,2854.75,1953.25,1652.75,1502.50,1953.25
Laptop Pro,1200.50,14406.00,16807.00,32413.50,9604.00,22809.50,7203.00,20408.50,8403.50,21609.00,16807.00,14406.00
Souris Gamer,1050.00,750.00,600.00,600.00,600.00,75.00,1050.00,975.00,375.00,450.00,375.00,750.00
Webcam HD,1399.86,999.90,1799.82,999.90,1499.85,1299.87,899.91,899.91,799.92,1099.89,699.93,599.94
Écran 4K,5105.94,4254.95,4254.95,15317.82,5105.94,11062.87,9360.89,7658.91,13615.84,10211.88,10211.88,4254.95


## Livrable

Vous devez soumettre ce notebook (`.ipynb`) complété. Le notebook doit s'exécuter sans erreur du début à la fin et afficher les DataFrames résultats de chaque analyse.